# Rating Product & Sorting Reviews in Amazon

In [1]:
# Libraries
import pandas as pd
import math
import scipy.stats as st

# Settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Task 1: 
Calculate the Average Rating according to the current comments and compare it with the existing Average Rating. 

In the shared dataset, users gave scores and comments to a product. In this task, our aim is to evaluate the given scores by weighting them by date. It is necessary to compare the first average score with the weighted score according to the date to be obtained.

Step 1: Read the data and calculate the average rate of the product.

In [2]:
df = pd.read_csv('amazon_review.csv')

In [3]:
def check_df(dataframe, head=7, tail=7):
    '''
    Prints the general information about the given dataframe e.g. shape, head,
    tail, info, descriptive statistics, etc.

    Parameters
    ----------
    dataframe : DataFrame
        The dataframe that we want to have general information about.
    head: int
        Prints the first n rows of the dataframe.
    tail: int
        Prints the last n rows of the dataframe.
    '''
    print('####### Shape #######')
    print(dataframe.shape)
    print('####### Info #######')
    print(dataframe.info())
    print('####### Head #######')
    print(dataframe.head(head))
    print('####### Tail #######')
    print(dataframe.tail(tail))
    print('####### Descriptive Statistics #######')
    print(dataframe.describe([0.05, 0.25, 0.50, 0.75 ,0.95, 0.99]).T)
    print('####### NA #######')
    print(dataframe.isnull().sum())
    print('####### Number of Unique Values #######')
    print(dataframe.nunique())


check_df(df)

####### Shape #######
(4915, 12)
####### Info #######
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4915 entries, 0 to 4914
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      4915 non-null   object 
 1   asin            4915 non-null   object 
 2   reviewerName    4914 non-null   object 
 3   helpful         4915 non-null   object 
 4   reviewText      4914 non-null   object 
 5   overall         4915 non-null   float64
 6   summary         4915 non-null   object 
 7   unixReviewTime  4915 non-null   int64  
 8   reviewTime      4915 non-null   object 
 9   day_diff        4915 non-null   int64  
 10  helpful_yes     4915 non-null   int64  
 11  total_vote      4915 non-null   int64  
dtypes: float64(1), int64(4), object(7)
memory usage: 460.9+ KB
None
####### Head #######
       reviewerID        asin  reviewerName helpful                                         reviewText  overall        

In [4]:
# Average rate of the product
df['overall'].mean()

4.587589013224822

Step 2: Calculate the weighted average of rating by date.

In [5]:
def time_based_weighted_average(dataframe, w1=30, w2=26, w3=24, w4=20):
    return dataframe.loc[df['day_diff'] <= 30, 'overall'].mean() * w1 / 100 + \
           dataframe.loc[(dataframe['day_diff'] > 30) & (dataframe['day_diff'] <= 90), 'overall'].mean() * w2 / 100 + \
           dataframe.loc[(dataframe['day_diff'] > 90) & (dataframe['day_diff'] <= 180), 'overall'].mean() * w3 / 100 + \
           dataframe.loc[(dataframe['day_diff'] > 180), 'overall'].mean() * w4 / 100


time_based_weighted_average(df)

4.702097124460948

# Task 2: 
Specify 20 reviews for the product to be displayed on the product detail page.

Step 1: Generate the 'helpful_no' variable. 
Note:
• 'total_vote' is the total number of up-downs given to a comment.
• 'up' means helpful.
• There is no 'helpful_no' variable in the dataset, it must be generated over existing variables.
• Find the number of votes not useful ('helpful_no') by subtracting the number of helpful votes ('helpful_yes') from the total number of votes ('total_vote').

In [6]:
df['helpful_no']= df['total_vote'] - df['helpful_yes']
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,helpful_no
0,A3SBTW3WS4IQSN,B007WTAJTO,NaN,"[0, 0]",No issues.,4.000,Four Stars,1406073600,2014-07-23,138,0,0,0
1,A18K1ODH1I2MVB,B007WTAJTO,0mie,"[0, 0]","Purchased this for my device, it worked as adv...",5.000,MOAR SPACE!!!,1382659200,2013-10-25,409,0,0,0
2,A2FII3I2MBMUIA,B007WTAJTO,1K3,"[0, 0]",it works as expected. I should have sprung for...,4.000,nothing to really say....,1356220800,2012-12-23,715,0,0,0
3,A3H99DFEG68SR,B007WTAJTO,1m2,"[0, 0]",This think has worked out great.Had a diff. br...,5.000,Great buy at this price!!! *** UPDATE,1384992000,2013-11-21,382,0,0,0
4,A375ZM4U047O79,B007WTAJTO,2&amp;1/2Men,"[0, 0]","Bought it with Retail Packaging, arrived legit...",5.000,best deal around,1373673600,2013-07-13,513,0,0,0


Step 2: Calculate 'score_pos_neg_diff', 'score_average_rating' and 'wilson_lower_bound' scores and add to data. 

Note:
- To calculate 'score_pos_neg_diff', 'score_average_rating' and 'wilson_lower_bound scores', define the 'score_pos_neg_diff', 'score_average_rating' and 'wilson_lower_bound' functions.
- Create scores based on 'score_pos_neg_diff'. Next; Save it as 'score_pos_neg_diff' in df.
- Create scores based on 'score_average_rating'. Next; Save it as 'score_average_rating' in df.
- Create scores according to 'wilson_lower_bound'. Next; Save it as 'wilson_lower_bound' in df.

In [7]:
# 'score_pos_neg_diff'
# Up-Down Diff Score = (up ratings) − (down ratings)
def score_up_down_diff(up, down):
    return up - down

In [8]:
# 'score_average_rating'
# Score = Average rating = (up ratings) / (all ratings)
def score_average_rating(up, down):
    if up + down == 0:
        return 0
    return up / (up + down)

In [9]:
# 'wilson_lower_bound scores' 
def wilson_lower_bound(up, down, confidence=0.95):
    '''
    Calculate Wilson Lower Bound Score

    - The lower limit of the confidence interval to be calculated for the Bernoulli parameter p is accepted as the WLB score.
    - The score to be calculated is used for product ranking.
    - Note:
    If the scores are between 1-5, 1-3 are marked as negative, 4-5 as positive and can be adjusted to bernoulli.
    This brings with it some problems. For this reason, it is necessary to make a bayesian average rating.

    parameters
    ----------
    up: int
        up count
    down: int
        down count
    confidence: float
        confidence

    Returns
    -------
    wilson score: float
    '''
    n = up + down
    if n == 0:
        return 0
    z = st.norm.ppf(1 - (1 - confidence) / 2)
    phat = 1.0 * up / n
    return (phat + z * z / (2 * n) - z * math.sqrt((phat * (1 - phat) + z * z / (4 * n)) / n)) / (1 + z * z / n)

In [10]:
# score_pos_neg_diff
df['score_pos_neg_diff'] = df.apply(lambda x: score_up_down_diff(x['helpful_yes'], x['helpful_no']), axis=1)

# score_average_rating
df['score_average_rating'] = df.apply(lambda x: score_average_rating(x['helpful_yes'], x['helpful_no']), axis=1)

# wilson_lower_bound
df['wilson_lower_bound'] = df.apply(lambda x: wilson_lower_bound(x['helpful_yes'], x['helpful_no']), axis=1)

Step 3: Identify and rank the top 20 comments according to 'wilson_lower_bound'.

In [11]:
df.sort_values('wilson_lower_bound', ascending=False)[:20]

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,day_diff,helpful_yes,total_vote,helpful_no,score_pos_neg_diff,score_average_rating,wilson_lower_bound
2031,A12B7ZMXFI6IXY,B007WTAJTO,"Hyoun Kim ""Faluzure""","[1952, 2020]",[[ UPDATE - 6/19/2014 ]]So my lovely wife boug...,5.000,UPDATED - Great w/ Galaxy S4 & Galaxy Tab 4 10...,1367366400,2013-01-05,702,1952,2020,68,1884,0.966,0.958
3449,AOEAD7DPLZE53,B007WTAJTO,NLee the Engineer,"[1428, 1505]",I have tested dozens of SDHC and micro-SDHC ca...,5.000,Top of the class among all (budget-priced) mic...,1348617600,2012-09-26,803,1428,1505,77,1351,0.949,0.937
4212,AVBMZZAFEKO58,B007WTAJTO,SkincareCEO,"[1568, 1694]",NOTE: please read the last update (scroll to ...,1.000,1 Star reviews - Micro SDXC card unmounts itse...,1375660800,2013-05-08,579,1568,1694,126,1442,0.926,0.912
317,A1ZQAQFYSXL5MQ,B007WTAJTO,"Amazon Customer ""Kelly""","[422, 495]","If your card gets hot enough to be painful, it...",1.000,"Warning, read this!",1346544000,2012-02-09,1033,422,495,73,349,0.853,0.819
4672,A2DKQQIZ793AV5,B007WTAJTO,Twister,"[45, 49]",Sandisk announcement of the first 128GB micro ...,5.000,Super high capacity!!! Excellent price (on Am...,1394150400,2014-07-03,158,45,49,4,41,0.918,0.808
1835,A1J6VSUM80UAF8,B007WTAJTO,goconfigure,"[60, 68]",Bought from BestBuy online the day it was anno...,5.000,I own it,1393545600,2014-02-28,283,60,68,8,52,0.882,0.785
3981,A1K91XXQ6ZEBQR,B007WTAJTO,"R. Sutton, Jr. ""RWSynergy""","[112, 139]",The last few days I have been diligently shopp...,5.000,"Resolving confusion between ""Mobile Ultra"" and...",1350864000,2012-10-22,777,112,139,27,85,0.806,0.732
3807,AFGRMORWY2QNX,B007WTAJTO,R. Heisler,"[22, 25]",I bought this card to replace a lost 16 gig in...,3.000,"Good buy for the money but wait, I had an issue!",1361923200,2013-02-27,649,22,25,3,19,0.880,0.700
4306,AOHXKM5URSKAB,B007WTAJTO,Stellar Eller,"[51, 65]","While I got this card as a ""deal of the day"" o...",5.000,Awesome Card!,1339200000,2012-09-06,823,51,65,14,37,0.785,0.670
4596,A1WTQUOQ4WG9AI,B007WTAJTO,"Tom Henriksen ""Doggy Diner""","[82, 109]",Hi:I ordered two card and they arrived the nex...,1.000,Designed incompatibility/Don't support SanDisk,1348272000,2012-09-22,807,82,109,27,55,0.752,0.664
